In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

In [8]:
data=pd.read_csv('Data.csv')
data.drop_duplicates(['RANK'],inplace=True)
# Combine 'BRANCH' and 'COLLEGE' columns into 'target'
data['target'] = data['COLLEGE'] + '_' + data['BRANCH']

# Drop 'CANDIDATE NAME' column
data.drop(['CANDIDATE NAME','COLLEGE','BRANCH',], axis=1, inplace=True)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
le=LabelEncoder()
data['target']=le.fit_transform(data['target'])
#columns_to_encode = ['GENDER', 'REGION', 'CATEGORY', 'SEAT CATEGORY']
columns_to_encode = ['SEAT CATEGORY','GENDER', 'REGION', 'CATEGORY']

print(data.head())
# Define the columns to be encoded and the columns to be skipped
transfor = Pipeline([
    ('encoding', ColumnTransformer([('one_hot_encoding', OneHotEncoder(), columns_to_encode)], remainder='passthrough'))
])

data_trans=transfor.fit_transform(data.drop('target',axis=1))

   RANK GENDER REGION CATEGORY SEAT CATEGORY  target
0   144      M     OU     BC_B     OC_GEN_UR     534
1   233      M     OU     BC_D     OC_GEN_UR     534
2   243      M     OU     BC_B   BC_B_GEN_UR     534
3   286      M     OU       OC    EWS_GEN_UR     534
4   318      M     OU     BC_D   BC_D_GEN_OU     291


In [10]:
data_trans

<49310x211 sparse matrix of type '<class 'numpy.float64'>'
	with 246550 stored elements in Compressed Sparse Row format>

In [23]:
model=Pipeline([("clf",KNeighborsClassifier())])

from skopt import BayesSearchCV
from skopt.space import Real,Integer

search_space = {
    'clf__algorithm': ['auto', 'brute'],
    'clf__leaf_size': Integer(1,100),
    'clf__n_neighbors': Integer(1,80),
    'clf__metric': ['cityblock', 'euclidean', 'l1', 'l2', 'manhattan','cosine'],
    'clf__p': Integer(0, 3),
    'clf__weights': ['uniform','distance'],}

opt = BayesSearchCV(model, search_space, cv=2, n_iter=10, scoring='accuracy') 

opt.fit(data_trans,data['target'])

c:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavi

BayesSearchCV(cv=2, estimator=Pipeline(steps=[('clf', KNeighborsClassifier())]),
              n_iter=10, scoring='accuracy',
              search_spaces={'clf__algorithm': ['auto', 'brute'],
                             'clf__leaf_size': Integer(low=1, high=100, prior='uniform', transform='normalize'),
                             'clf__metric': ['cityblock', 'euclidean', 'l1',
                                             'l2', 'manhattan', 'cosine'],
                             'clf__n_neighbors': Integer(low=1, high=80, prior='uniform', transform='normalize'),
                             'clf__p': Integer(low=0, high=3, prior='uniform', transform='normalize'),
                             'clf__weights': ['uniform', 'distance']})

In [25]:
best=opt.best_estimator_
best.fit(data_trans,data['target'])

Pipeline(steps=[('clf',
                 KNeighborsClassifier(leaf_size=20, metric='cosine',
                                      n_neighbors=39, p=0))])

Pipeline(steps=[('clf',
                 KNeighborsClassifier(leaf_size=20, metric='cosine',
                                      n_neighbors=39, p=0))])

In [32]:
test=[[25000,'F','OU','BC_C','BC_C_GIRLS_OU']]  #20k rank, girl
test=pd.DataFrame(test,columns=['RANK','GENDER','REGION','CATEGORY','SEAT CATEGORY'])
test_trans=transfor.transform(test)
print(test_trans[0])
le.inverse_transform( best.predict(test_trans) )

  (0, 50)	1.0
  (0, 196)	1.0
  (0, 200)	1.0
  (0, 204)	1.0
  (0, 210)	25000.0


c:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


array(['GCTC_CSE'], dtype=object)

In [33]:
lofpro=list(best.predict_proba(test_trans))
for i in range(5):
    max=np.argmax(lofpro)
    print(f'{le.inverse_transform([max])} {lofpro[0][max]*100} % probable admission')
    lofpro[0]=np.delete(lofpro[0],max)

['GCTC_CSE'] 5.128205128205128 % probable admission
['BREW_CSE'] 2.564102564102564 % probable admission
['BREW_EEE'] 2.564102564102564 % probable admission
['BVRI_MEC'] 2.564102564102564 % probable admission
['BVRI_MEC'] 2.564102564102564 % probable admission


In [37]:
le.inverse_transform([0])

array(['AARM_CSE'], dtype=object)